In [2]:
from base import *
from utils import pauli_sum, entropy
from plot_tools import *
import qiskit as qk
from qiskit import QuantumCircuit, Aer, execute
from qiskit.primitives import Estimator
# from qiskit.utils import QuantumInstance
# from qiskit.algorithms import VQE
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP, ADAM, COBYLA
from qiskit.opflow import X, Z, I, Y, AerPauliExpectation, PauliSumOp
from qiskit.circuit import Parameter 
from qiskit.circuit.library import TwoLocal
import numpy as np

In [3]:
from algorithms import Vqe 

## J=2

In [14]:
# constants
eps = 1
z_coeff = 0.5 * eps



In [15]:

def H(V,W):
    return np.array([[-2*eps, 0, np.sqrt(6)*V, 0, 0],
                  [0, -eps+3*W, 0, 3*V, 0],
                  [np.sqrt(6)*V, 4*W, 0, 0, np.sqrt(6)*V],
                  [0, 3*V, 0, eps+3*W, 0],
                  [0, 0, np.sqrt(6)*V, 0, 2*eps]])

# H(0)

# def my_H(v):
    
#     x_coeff = -v/2
#     y_coeff = v/2
#     return pauli_sum([('ZIII', z_coeff), ('IZII', z_coeff), ('IIZI', z_coeff), ('IIIZ', z_coeff), # all 1 qubit combinations
#                       ('XXII', x_coeff), ('XIXI', x_coeff), ('XIIX', x_coeff), ('IXXI', x_coeff), ('IXIX', x_coeff), ('IIXX', x_coeff), # all 2 qubit combinations
#                       ('YYII', y_coeff), ('YIYI', y_coeff), ('YIIY', y_coeff), ('IYYI', y_coeff), ('IYIY', y_coeff), ('IIYY', y_coeff)])

# my_H(0)

In [26]:
vs = np.arange(0, 2, 0.02)
# ws = np.arange(0, 2, 0.02)
eigs = np.zeros((5, len(vs)))

# let v=w
for i, v in enumerate(vs):
    eigvals, eigvecs = np.linalg.eig(H(v,v))
    eigs[:,i] = sorted(eigvals)

In [28]:
plot_2D(vs, eigs, plot_count=5, title=r"Numerical Eigenvalues for $J=2$ with v=w", x_title=r'$v$', 
        label = [rf'$E_{i}$' for i in range(5)], y_title='Energies for v=w', filename='lipkin-eigs-4-w.pdf')
        #  filename='lipkin-eigs-4.pdf')

As we can see, every eigenvalue has a degenerate value. However $E_3$ and $E_{14}$ are energy eigenvalues that we do not get from the 5x5 matrix. 

In [29]:
def ansatz(num_qubits, reps):
    # Create quantum circuit
    qc = QuantumCircuit(num_qubits)

    # Define parameters
    params = [Parameter(f'theta_{i}') for i in range(2 * num_qubits * (reps+1))]

    # Create the layers
    for i in range(reps): 
        # Add ry layer
        for q in range(num_qubits):
            qc.ry(params[2*(i*num_qubits + q)], q)
        # Add rz layer
        for q in range(num_qubits):
            qc.rx(params[2*(i*num_qubits + q) + 1], q)
        # Add entanglement layer (full entanglement: cz gates between every pair)
        for source in range(num_qubits):
            for target in range(source + 1, num_qubits):
                qc.cx(source, target)

    return qc


